In [1]:
import torch
import torch.onnx as onnx

In [2]:
import models
import json
RATIO_DATA = 'data/ratio_data.json'
ADVICE_MODEL = 'advice_resnet18_aug_1'
ADVICE_ONE_MODEL = 'advice_one_resnet18_aug_1'
ADVICE_TWO_MODEL = 'advice_two_resnet18_aug_2'
ENCHANT_N_MODEL = 'enchant_n_resnet18_aug_1'
OPTION_MODEL = 'option_resnet18_aug_1'

with open(f'logs/{ADVICE_MODEL}/config.json') as f:
    advice_config = json.load(f)
advice_model = getattr(models, advice_config['model_name'])(**advice_config['model_kwargs'])
advice_model.load_state_dict(torch.load(f'logs/{ADVICE_MODEL}/checkpoints/best_acc.pt'))
advice_model = advice_model.eval()

with open(f'logs/{ADVICE_ONE_MODEL}/config.json') as f:
    advice_one_config = json.load(f)
advice_one_model = getattr(models, advice_one_config['model_name'])(**advice_one_config['model_kwargs'])
advice_one_model.load_state_dict(torch.load(f'logs/{ADVICE_ONE_MODEL}/checkpoints/best_acc.pt'))
advice_one_model = advice_one_model.eval()

with open(f'logs/{ADVICE_TWO_MODEL}/config.json') as f:
    advice_two_config = json.load(f)
advice_two_model = getattr(models, advice_two_config['model_name'])(**advice_two_config['model_kwargs'])
advice_two_model.load_state_dict(torch.load(f'logs/{ADVICE_TWO_MODEL}/checkpoints/best_acc.pt'))
advice_two_model = advice_two_model.eval()

with open(f'logs/{ENCHANT_N_MODEL}/config.json') as f:
    enchant_n_config = json.load(f)
enchant_n_model = getattr(models, enchant_n_config['model_name'])(**enchant_n_config['model_kwargs'])
enchant_n_model.load_state_dict(torch.load(f'logs/{ENCHANT_N_MODEL}/checkpoints/best_acc.pt'))
enchant_n_model = enchant_n_model.eval()

with open(f'logs/{OPTION_MODEL}/config.json') as f:
    option_config = json.load(f)
option_model = getattr(models, option_config['model_name'])(**option_config['model_kwargs'])
option_model.load_state_dict(torch.load(f'logs/{OPTION_MODEL}/checkpoints/best_acc.pt'))
option_model = option_model.eval()


In [17]:
advice_sample = torch.randn(1, 3, 64, 288)
# enchants_sample = torch.randn(1, 3, 16, 32, requires_grad=True)
# options_sample = torch.randn(1, 3, 20, 105, requires_grad=True)
with torch.no_grad():
    advice_output = advice_model(advice_sample)
torch.onnx.export(
    advice_model,
    advice_sample,
    'onnx_models/advice_resnet18_aug_1.onnx',
    export_params=True,
    opset_version=16,
    verbose=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    }
)

In [7]:
import onnxruntime as ort
import numpy as np
ort_session = ort.InferenceSession('onnx_models/advice_resnet18_aug_1.onnx')
ort_inputs = {ort_session.get_inputs()[0].name: np.random.random((10, 3, 64, 288)).astype(np.float32)}
ort_outs = ort_session.run(None, ort_inputs)
# np.testing.assert_allclose(advice_output.detach().numpy(), ort_outs[0], rtol=1e-03, atol=1e-05)
    

In [8]:
ort_outs[0].shape


(10, 132)

In [21]:
advice_one_sample = torch.randn(1, 3, 64, 288)
with torch.no_grad():
    advice_one_output = advice_one_model(advice_one_sample)
torch.onnx.export(
    advice_one_model,
    advice_one_sample,
    'onnx_models/advice_one_resnet18_aug_1.onnx',
    export_params=True,
    opset_version=16,
    verbose=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    }
)

ort_session = ort.InferenceSession('onnx_models/advice_one_resnet18_aug_1.onnx')
ort_inputs = {ort_session.get_inputs()[0].name: advice_one_sample.detach().numpy()}
ort_outs = ort_session.run(None, ort_inputs)
np.testing.assert_allclose(advice_one_output.detach().numpy(), ort_outs[0], rtol=1e-03, atol=1e-05)

print(advice_one_output.detach().numpy()[0][:10])
print(ort_outs[0][0][:10])

[-319.90106 -305.30865 -275.81003 -297.64792 -221.13457 -210.89893
 -181.3021  -224.26352 -195.546   -245.26265]
[-319.901   -305.3086  -275.80994 -297.64792 -221.13452 -210.8989
 -181.3021  -224.26355 -195.54602 -245.26263]


In [22]:
advice_two_sample = torch.randn(1, 3, 64, 288)
with torch.no_grad():
    advice_two_output = advice_two_model(advice_two_sample)
torch.onnx.export(
    advice_two_model,
    advice_two_sample,
    'onnx_models/advice_two_resnet18_aug_2.onnx',
    export_params=True,
    opset_version=16,
    verbose=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    }
)

ort_session = ort.InferenceSession('onnx_models/advice_two_resnet18_aug_2.onnx')
ort_inputs = {ort_session.get_inputs()[0].name: advice_two_sample.detach().numpy()}
ort_outs = ort_session.run(None, ort_inputs)
np.testing.assert_allclose(advice_two_output.detach().numpy(), ort_outs[0], rtol=1e-03, atol=1e-05)

print(advice_two_output.detach().numpy()[0][:10])
print(ort_outs[0][0][:10])

[-49.20324  -48.14781  -47.634052 -55.616154 -66.9009   -39.733562
 -45.83955  -57.330097 -49.242924 -55.50102 ]
[-49.203224 -48.147804 -47.63404  -55.616142 -66.900894 -39.73356
 -45.83953  -57.330093 -49.242916 -55.501007]


In [23]:
enchant_n_sample = torch.randn(1, 3, 16, 32)
with torch.no_grad():
    enchant_n_output = enchant_n_model(enchant_n_sample)
torch.onnx.export(
    enchant_n_model,
    enchant_n_sample,
    'onnx_models/enchant_n_resnet18_aug_1.onnx',
    export_params=True,
    opset_version=16,
    verbose=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    }
)

ort_session = ort.InferenceSession('onnx_models/enchant_n_resnet18_aug_1.onnx')
ort_inputs = {ort_session.get_inputs()[0].name: enchant_n_sample.detach().numpy()}
ort_outs = ort_session.run(None, ort_inputs)
np.testing.assert_allclose(enchant_n_output.detach().numpy(), ort_outs[0], rtol=1e-03, atol=1e-05)

print(enchant_n_output.detach().numpy()[0][:10])
print(ort_outs[0][0][:10])

[-0.42014518 -2.271747   -2.8423336  -0.9598536   5.0915184   3.7943265
 -3.5288858  -0.6669427  -0.33164066 -3.078349  ]
[-0.42014426 -2.2717438  -2.8423324  -0.95985496  5.0915127   3.7943254
 -3.5288806  -0.6669439  -0.3316393  -3.078349  ]


In [24]:
option_sample = torch.randn(1, 3, 20, 105)
with torch.no_grad():
    option_output = option_model(option_sample)
torch.onnx.export(
    option_model,
    option_sample,
    'onnx_models/option_resnet18_aug_1.onnx',
    export_params=True,
    opset_version=16,
    verbose=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    }
)

ort_session = ort.InferenceSession('onnx_models/option_resnet18_aug_1.onnx')
ort_inputs = {ort_session.get_inputs()[0].name: option_sample.detach().numpy()}
ort_outs = ort_session.run(None, ort_inputs)
np.testing.assert_allclose(option_output.detach().numpy(), ort_outs[0], rtol=1e-03, atol=1e-05)

print(option_output.detach().numpy()[0][:10])
print(ort_outs[0][0][:10])

[-11.827245   -10.277574    -9.819902   -14.344846    -0.02018761
   0.2792394    1.075137    -8.602564    -2.5980327    0.2627149 ]
[-11.827242   -10.27757     -9.819905   -14.344847    -0.02018636
   0.27923828   1.0751365   -8.602562    -2.5980315    0.26271605]
